In [1]:
import gensim
from gensim.models import KeyedVectors
import numpy as np
import pandas as pd

Импортируем обученные Word2Vec вектора:

In [2]:
path_to_word_vectors = '~/Desktop/Projects/Data/GoogleNews/GoogleNews-vectors-negative300.bin.gz'
model = KeyedVectors.load_word2vec_format(path_to_word_vectors, binary=True)

Посмотрим, какой размер имеют вектора:

In [25]:
model.vector_size

300

Посмотрим, как выглядит вектор слова 'man':

In [3]:
vec = model['man']
print(vec)

[ 0.32617188  0.13085938  0.03466797 -0.08300781  0.08984375 -0.04125977
 -0.19824219  0.00689697  0.14355469  0.0019455   0.02880859 -0.25
 -0.08398438 -0.15136719 -0.10205078  0.04077148 -0.09765625  0.05932617
  0.02978516 -0.10058594 -0.13085938  0.001297    0.02612305 -0.27148438
  0.06396484 -0.19140625 -0.078125    0.25976562  0.375      -0.04541016
  0.16210938  0.13671875 -0.06396484 -0.02062988 -0.09667969  0.25390625
  0.24804688 -0.12695312  0.07177734  0.3203125   0.03149414 -0.03857422
  0.21191406 -0.00811768  0.22265625 -0.13476562 -0.07617188  0.01049805
 -0.05175781  0.03808594 -0.13378906  0.125       0.0559082  -0.18261719
  0.08154297 -0.08447266 -0.07763672 -0.04345703  0.08105469 -0.01092529
  0.17480469  0.30664062 -0.04321289 -0.01416016  0.09082031 -0.00927734
 -0.03442383 -0.11523438  0.12451172 -0.0246582   0.08544922  0.14355469
 -0.27734375  0.03662109 -0.11035156  0.13085938 -0.01721191 -0.08056641
 -0.00708008 -0.02954102  0.30078125 -0.09033203  0.03149

Проверим, правильно ли модель находит похожиие слова для слова:

In [5]:
model.most_similar('man')

[('woman', 0.7664012908935547),
 ('boy', 0.6824870705604553),
 ('teenager', 0.6586930155754089),
 ('teenage_girl', 0.6147903203964233),
 ('girl', 0.5921714305877686),
 ('suspected_purse_snatcher', 0.5716364979743958),
 ('robber', 0.5585118532180786),
 ('Robbery_suspect', 0.5584409832954407),
 ('teen_ager', 0.5549196004867554),
 ('men', 0.5489763021469116)]

Способ подсчета косинусного расстояния между векторами:

In [24]:
list(model.cosine_similarities(model['man'], [model['woman'], model['rat']]))

[0.76640123, 0.21210901]

Вычисление вектора текста:

In [27]:
def count_bag_of_words(model: gensim.models.keyedvectors.KeyedVectors, text: str) -> np.ndarray:
    words = text.split(' ')
    text_vector = np.zeros(model.vector_size)
    indexed_words_quantity = 0
    for word in words:
        if model.has_index_for(word):
            indexed_words_quantity += 1
            text_vector += model[word]
    if indexed_words_quantity != 0:
        text_vector /= indexed_words_quantity

    return text_vector

Нахождение самых близких к запросу текстов

In [13]:
def find_most_similar_text(model: gensim.models.keyedvectors.KeyedVectors, query: str, texts: list[str], top_k: int) -> list[str]:
    '''

    Args:
        model (gensim.models.keyedvectors.KeyedVectors): Word vectors
        query (str): User query
        texts (list[str]): list of arts descriptions
        top_k (int): Hom many arts you want

    Returns:
        list[str]: List of most similar descriptions
    '''

    query_vector = count_bag_of_words(model=model, text=query)
    texts_vectors = pd.DataFrame({'text': texts, 
                                  'text_vector': list(map(lambda text: count_bag_of_words(model=model, text=text), texts))})
    texts_vectors['query_distance'] = list(model.cosine_similarities(query_vector, list(texts_vectors['text_vector'])))
    return texts_vectors.nlargest(n = top_k, columns='query_distance')['text'].to_list()

In [29]:
assert find_most_similar_text(model= model, query='impressionism',texts = ['man in black', 'starry night'], top_k = 1) == ['starry night']

In [17]:
df.head()

,artwork_id,title,medium,department,classification,size_category,duration_category,release_date,name,nationality,gender,birth_year,death_year,feature_joined_text
0,0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Ink and cut-and-pasted painted pages on paper,Architecture & Design,Architecture,big,NaN,1896.0,Otto Wagner,Austrian,male,1841.0,1918.0,ferdinandsbrücke project vienna austria elevat...
1,1,"City of Music, National Superior Conservatory ...",Paint and colored pencil on print,Architecture & Design,Architecture,medium,NaN,1987.0,Christian de Portzamparc,French,male,1944.0,NaN,city of music national superior conservatory o...
2,2,"Villa near Vienna Project, Outside Vienna, Aus...","Graphite, pen, color pencil, ink, and gouache ...",Architecture & Design,Architecture,medium,NaN,1903.0,Emil Hoppe,Austrian,male,1876.0,1957.0,villa near vienna project outside vienna austr...
3,3,"The Manhattan Transcripts Project, New York, N...",Photographic reproduction with colored synthet...,Architecture & Design,Architecture,medium,NaN,1980.0,Bernard Tschumi,NaN,male,1944.0,NaN,the manhattan transcripts project new york new...
4,4,"Villa, project, outside Vienna, Austria, Exter...","Graphite, color pencil, ink, and gouache on tr...",Architecture & Design,Architecture,medium,NaN,1903.0,Emil Hoppe,Austrian,male,1876.0,1957.0,villa project outside vienna austria exterior ...
